In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import nltk, re, time
from nltk.corpus import stopwords
from string import punctuation
from collections import defaultdict
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from collections import namedtuple
from tensorflow.python.framework import ops
import tensorflow_addons as tfa

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [2]:
# Load the data
train = pd.read_csv("/kaggle/input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip", delimiter="\t")
test = pd.read_csv("/kaggle/input/word2vec-nlp-tutorial/testData.tsv.zip", delimiter="\t")

In [3]:
# Compare the lengths of the datasets
print(train.shape)
print(test.shape)

(25000, 3)
(25000, 2)


In [4]:
train.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [5]:
test.head()

,id,review
0,12311_10,Naturally in a film who's main themes are of m...
1,8348_2,This movie is a disaster within a disaster fil...
2,5828_4,"All in all, this is a movie for kids. We saw i..."
3,7186_2,Afraid of the Dark left me with the impression...
4,12128_7,A very accurate depiction of small time mob li...


In [6]:
 #Check for any null values
print(train.isnull().sum())
print(test.isnull().sum())

id           0
sentiment    0
review       0
dtype: int64
id        0
review    0
dtype: int64


In [7]:
def clean_text(text, remove_stopwords=True):
    '''Clean the text, with the option to remove stopwords'''
    
    # Convert words to lower case and split them
    text = text.lower().split()

    # Optionally, remove stop words
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
    
    text = " ".join(text)

    # Clean the text
    text = re.sub(r"<br />", " ", text)
    text = re.sub(r"[^a-z]", " ", text)
    text = re.sub(r"   ", " ", text) # Remove any extra spaces
    text = re.sub(r"  ", " ", text)
    
    # Remove punctuation from text
    text = ''.join([c for c in text if c not in punctuation])
    
    # Return a list of words
    return(text)

In [8]:
train_clean = []
for review in train.review:
    train_clean.append(clean_text(review))

In [9]:
test_clean = []
for review in test.review:
    test_clean.append(clean_text(review))

In [10]:
for i in range(3):
    print(train_clean[i])
    print()

stuff going moment mj i ve started listening music watching odd documentary there watched wiz watched moonwalker again maybe want get certain insight guy thought really cool eighties maybe make mind whether guilty innocent moonwalker part biography part feature film remember going see cinema originally released subtle messages mj s feeling towards press also obvious message drugs bad m kay visually impressive course michael jackson unless remotely like mj anyway going hate find boring may call mj egotist consenting making movie mj fans would say made fans true really nice him the actual feature film bit finally starts minutes excluding smooth criminal sequence joe pesci convincing psychopathic powerful drug lord wants mj dead bad beyond me mj overheard plans nah joe pesci s character ranted wanted people know supplying drugs etc dunno maybe hates mj s music lots cool things like mj turning car robot whole speed demon sequence also director must patience saint came filming kiddy bad seq

In [11]:
# Tokenize the reviews
all_reviews = train_clean + test_clean
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_reviews)
print("Fitting is complete.")

train_seq = tokenizer.texts_to_sequences(train_clean)
print("train_seq is complete.")

test_seq = tokenizer.texts_to_sequences(test_clean)
print("test_seq is complete")

Fitting is complete.
train_seq is complete.
test_seq is complete


In [12]:
# Find the number of unique tokens
word_index = tokenizer.word_index
print("Words in index: %d" % len(word_index))

Words in index: 99425


In [13]:
# Inspect the reviews after they have been tokenized
for i in range(3):
    print(train_seq[i])
    print()

[437, 81, 481, 10863, 6, 71, 573, 2590, 115, 65, 948, 551, 51, 207, 24383, 207, 17034, 213, 188, 92, 19, 684, 2550, 118, 104, 14, 511, 3933, 188, 25, 240, 644, 2336, 1251, 17034, 85, 4772, 85, 701, 3, 298, 81, 15, 351, 1827, 533, 1209, 3566, 10863, 1, 477, 861, 3526, 22, 517, 662, 1403, 18, 60, 5290, 2073, 1109, 180, 406, 1512, 807, 2559, 5, 10863, 469, 81, 655, 80, 265, 109, 569, 10863, 33435, 29469, 141, 2, 10863, 374, 12, 57, 24, 374, 205, 14, 246, 173, 9, 740, 701, 3, 135, 334, 456, 138, 16333, 4102, 1702, 626, 865, 10467, 1009, 11946, 880, 1058, 1640, 408, 10863, 258, 18, 584, 134, 10863, 17899, 2288, 15681, 865, 10467, 1, 32, 36026, 381, 20, 46, 17447, 1403, 426, 9779, 188, 4220, 10863, 1, 115, 658, 511, 91, 5, 10863, 1541, 436, 2257, 131, 2124, 2367, 626, 22, 67, 112, 4731, 5337, 300, 1313, 29470, 18, 626, 547, 878, 655, 685, 4, 444, 190, 537, 131, 677, 3371, 1224, 779, 54, 1229, 260, 2, 20, 5, 10863, 4, 570, 73, 468, 30, 20, 239, 695, 151, 269, 108, 7617, 662, 3514, 10863, 1, 3

In [14]:
# Find the length of reviews
lengths = []
for review in train_seq:
    lengths.append(len(review))

for review in test_seq:
    lengths.append(len(review))

# Create a dataframe so that the values can be inspected
lengths = pd.DataFrame(lengths, columns=['counts'])

In [15]:
lengths.counts.describe()

count    50000.000000
mean       128.366580
std         96.950296
min          3.000000
25%         68.000000
50%         95.000000
75%        156.000000
max       1488.000000
Name: counts, dtype: float64

In [16]:
print(np.percentile(lengths.counts, 80))
print(np.percentile(lengths.counts, 85))
print(np.percentile(lengths.counts, 90))
print(np.percentile(lengths.counts, 95))

178.0
208.0
253.0
332.0


In [17]:
# Pad and truncate the questions so that they all have the same length.
max_review_length = 250

train_pad = pad_sequences(train_seq, maxlen = max_review_length)
print("train_pad is complete.")

test_pad = pad_sequences(test_seq, maxlen = max_review_length)
print("test_pad is complete.")

train_pad is complete.
test_pad is complete.


In [18]:
# Inspect the reviews after padding has been completed. 
for i in range(3):
    print(train_pad[i,:100])
    print()

[    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0   437    81   481 10863     6    71   573  2590   115
    65   948   551    51   207 24383   207 17034   213   188    92    19
   684  2550   118   104    14   511  3933   188    25   240   644  2336
  1251 17034    85  4772    85   701     3   298    81    15   351  1827
   533  1209  3566 10863     1   477   861  3526    22   517   662  1403
    18    60  5290  2073  1109   180   406  1512   807  2559     5 10863
   469    81   655    80   265   109   569 10863 33435 29469   141     2
 10863   374    12    57]

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]

[    0     0     0     0     3   456  2875  4773  3612   659  2478 17035
   528  4475 10238  1147   928  1053 31269 10238  1626   707 50267 10239
     5 50268  1147  3015 11298     4  

In [19]:
# Creating the training and validation sets
x_train, x_valid, y_train, y_valid = train_test_split(train_pad, train.sentiment, test_size = 0.15, random_state = 2)
x_test = test_pad

In [20]:
# Inspect the shape of the data
print(x_train.shape)
print(x_valid.shape)
print(x_test.shape)

(21250, 250)
(3750, 250)
(25000, 250)


# **KNN MODEL BUILD & TRAIN**

In [21]:
def get_batches(x, y, batch_size):
    '''Create the batches for the training and validation data'''
    n_batches = len(x)//batch_size
    x, y = x[:n_batches*batch_size], y[:n_batches*batch_size]
    for ii in range(0, len(x), batch_size):
        yield x[ii:ii+batch_size], y[ii:ii+batch_size]

In [22]:
def get_test_batches(x, batch_size):
    '''Create the batches for the testing data'''
    n_batches = len(x)//batch_size
    x = x[:n_batches*batch_size]
    for ii in range(0, len(x), batch_size):
        yield x[ii:ii+batch_size]

In [23]:
# The default parameters of the model
n_words = len(word_index)
embed_size = 300
batch_size = 250
lstm_size = 128
num_layers = 2
dropout = 0.5
learning_rate = 0.001
epochs = 100
multiple_fc = False
fc_units = 256